# Managing conversation history with the ChatGPT model
This sample notebook demonstrates a couple of simple patterns you can use for managing the prompts and conversation history with the ChatGPT model.

In [2]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
!pip install --upgrade openai

  Attempting uninstall: openai
    Found existing installation: openai 1.60.1
    Uninstalling openai-1.60.1:
      Successfully uninstalled openai-1.60.1


In [12]:
# import os module & the OpenAI Python library for calling the OpenAI API
# please make sure you have installed required libraries via pip install -r requirements.txt
import os, dotenv
from openai import AzureOpenAI

dotenv.load_dotenv()

# Setting up the deployment name
deployment_name = "gpt-4o-mini"

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
# This is the value of the endpoint for your Azure OpenAI resource
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

# The API key for your Azure OpenAI resource.
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# Currently OPENAI API have the following versions available: https://learn.microsoft.com/azure/ai-services/openai/reference
api_version = "2024-07-01-preview"

client = AzureOpenAI(
  azure_endpoint = azure_endpoint,
  api_key = api_key,
  api_version = "2024-08-01-preview"  # This API version or later is required to access seed/events/checkpoint features
)

## 1.0 Create the system message for ChatGPT

In [13]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"
print(system_message)

You are a helpful assistant.


## 2.0 Define helper functions



In [4]:
!pip install tiktoken

In [62]:
import tiktoken

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [84]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, model_name, max_response_tokens=500, temperature=0):
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        max_tokens=max_response_tokens,
        temperature=temperature
    )
    return response.choices[0].message.content

# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()

## 3.0 Start the conversation

In [64]:
# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "I want to write a blog post about the impact of AI on the future of work."

In [65]:
# Create the list of messages. role can be either "user" or "assistant" 
messages = [
    {"role": "user", "content": "Hello, how are you?"},
    {"role": "assistant", "content": "I'm good, thank you! How can I assist you today?"},
    {"role": "user", "content": "Can you help me with a Python problem?"},
    {"role": "assistant", "content": "Of course! Please tell me more about the problem you're facing."}
]
print_conversation(messages)

[USER]
Hello, how are you?

[ASSISTANT]
I'm good, thank you! How can I assist you today?

[USER]
Can you help me with a Python problem?

[ASSISTANT]
Of course! Please tell me more about the problem you're facing.



In [66]:
token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

Token count: 65


In [67]:
max_response_tokens = 100

response = send_message(messages, deployment_name, max_response_tokens)
messages.append({"role": "assistant", "content": response})

print_conversation(messages)

[USER]
Hello, how are you?

[ASSISTANT]
I'm good, thank you! How can I assist you today?

[USER]
Can you help me with a Python problem?

[ASSISTANT]
Of course! Please tell me more about the problem you're facing.

[ASSISTANT]
Of course! Please let me know what specific Python problem you're working on, and I'll do my best to help you.



## 4.0 Continue the conversation

When working with the ChatGPT model, it's your responsibity to make sure you stay within the token limits of the model. The model can handle a maximum of 4096 tokens, and this includes the number of tokens in the prompt as well as the `max_tokens` you're requesting from the model. If you exceed these limits, the model will return an error.

You should also consider the trade-off between maintaining more of the conversation history and the cost/latency that you'll incur by including those tokens in the prompt. Shorter prompts are cheaper and faster. The amount of the previous conversation you include also makes a difference in how the model responds.

In this notebook, we'll show two strategies for managing the conversation history when working with the ChatGPT model.
- Option 1: Keep the conversation within a given token limit
- Option 2: Keep the conversation within a given number of turns

### Option 1: Keep the conversation within a given token limit

`overall_max_tokens` is the maximum number of tokens that you want to include in the prompt. Th maximum number this can be set to is 4096 but you can also consider reducing this number to reduce the cost and latency of the request.

In [68]:
overall_max_tokens = 1096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [69]:
user_message = "The target audience for the blog post should be business leaders working in the tech industry."
#user_message = "Let's talk about generative AI and keep the tone informational but also friendly."
#user_message = "Show me a few more examples"
messages.append({"role": "user", "content": user_message})

token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

# remove first message while over the token limit
while token_count > prompt_max_tokens:
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = send_message(messages, deployment_name)

#Append assistance response 
messages.append({"role": "assistant", "content": response})
print_conversation(messages)

Token count: 118
[USER]
Hello, how are you?

[ASSISTANT]
I'm good, thank you! How can I assist you today?

[USER]
Can you help me with a Python problem?

[ASSISTANT]
Of course! Please tell me more about the problem you're facing.

[ASSISTANT]
Of course! Please let me know what specific Python problem you're working on, and I'll do my best to help you.

[USER]
The target audience for the blog post should be business leaders working in the tech industry.

[ASSISTANT]
That sounds great! If you're writing a blog post aimed at business leaders in the tech industry, you'll want to consider topics that are relevant and valuable to them. Here are a few ideas you could explore:

### Potential Blog Post Topics:

1. **The Future of Work in Tech:** Explore how remote and hybrid work models are shaping the tech industry and what leaders should consider when adapting their strategies.

2. **Leveraging AI for Business Growth:** Discuss how artificial intelligence can be integrated into business proce

### Option 2: Keep the conversation within a given number of turns

In [70]:
max_messages = 10

overall_max_tokens = 4096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [74]:
user_message = "Make the post about generative AI aimed at business leaders who have some knowledge of the technology."
messages.append({"role": "user", "content": user_message})

while len(messages) > max_messages:
    messages.pop(0)

token_count = num_tokens_from_messages(messages)

while token_count > prompt_max_tokens:
    # remove first message from messages
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = send_message(messages, deployment_name, max_response_tokens)

#Append assistant response
messages.append({"role": "user", "content": response})
print_conversation(messages)


[USER]
What are the current trends in the tech stock market?

[ASSISTANT]
As of my last knowledge update in October 2023, I can provide some general trends that have been observed in the tech stock market leading up to that point. However, for the most current insights, I recommend checking real-time financial news and stock market analysis. Here are some notable trends that were observed:

1. **Artificial Intelligence (AI) Boom**: Companies developing AI technologies, particularly those involved in machine learning, natural language processing, and generative AI, saw significant increases in investment and stock prices. Major players like NVIDIA and Microsoft, which have been heavily involved in AI, benefited greatly.

2. **Cloud Computing Growth**: Cloud services continued to be a strong driver of growth for tech companies. Firms that offer cloud infrastructure and services (like Amazon AWS, Microsoft Azure, and Google Cloud) have been performing well.

3. **Increased Focus on Cybers

## Ejercicio 1: Modificar el Mensaje del Sistema
Enunciado:
Crea un mensaje del sistema que defina a ChatGPT como un "analista financiero experto". Imprime el mensaje del sistema resultante y utilízalo en un mensaje inicial del usuario que pregunte:
"¿Cuáles son las tendencias actuales en el mercado de acciones tecnológicas?".


In [73]:

# Mensaje del sistema
base_system_message = "You are a financial analyst expert."
system_message = f"{base_system_message.strip()}"
print(system_message)

# Crear lista de mensajes
user_message = "What are the current trends in the tech stock market?"
messages = [
    {"role": "user", "content": user_message},
]

# Enviar mensaje y recibir respuesta
response = send_message(messages, deployment_name)

messages.append({"role": "assistant", "content": response})
print_conversation(messages)

print(messages)


You are a financial analyst expert.
[USER]
What are the current trends in the tech stock market?

[ASSISTANT]
As of my last knowledge update in October 2023, I can provide some general trends that have been observed in the tech stock market leading up to that point. However, for the most current insights, I recommend checking real-time financial news and stock market analysis. Here are some notable trends that were observed:

1. **Artificial Intelligence (AI) Boom**: Companies developing AI technologies, particularly those involved in machine learning, natural language processing, and generative AI, saw significant increases in investment and stock prices. Major players like NVIDIA and Microsoft, which have been heavily involved in AI, benefited greatly.

2. **Cloud Computing Growth**: Cloud services continued to be a strong driver of growth for tech companies. Firms that offer cloud infrastructure and services (like Amazon AWS, Microsoft Azure, and Google Cloud) have been performing w

## Ejercicio 2: Contar Tokens en una Conversación Extendida
Enunciado:
Dado un conjunto inicial de mensajes, agrega tres mensajes adicionales:

"Explain the impact of inflation on stock prices."
"What are the top stocks to watch in 2025?"
"Summarize the latest market news in 50 words."
Calcula y muestra el número total de tokens tras añadir cada mensaje.


In [75]:

# Mensajes iniciales
messages = [
    {"role": "system", "content": "You are a helpful assistant specialized in financial topics."},
    {"role": "user", "content": "What are the current trends in the tech stock market?"}
]

# Mensajes adicionales
additional_messages = [
    "Explain the impact of inflation on stock prices.",
    "What are the top stocks to watch in 2025?",
    "Summarize the latest market news in 50 words."
]

# Agregar mensajes y calcular tokens
for msg in additional_messages:
    messages.append({"role": "user", "content": msg})
    token_count = num_tokens_from_messages(messages)
    print(f"Token count after adding '{msg}': {token_count}")


Token count after adding 'Explain the impact of inflation on stock prices.': 48
Token count after adding 'What are the top stocks to watch in 2025?': 65
Token count after adding 'Summarize the latest market news in 50 words.': 82


## Ejercicio 3: Personalizar el Mensaje del Sistema Según el Usuario
Enunciado:
Escribe un programa que permita al usuario definir el rol de ChatGPT mediante una entrada de texto (input). Una vez definido, responde al mensaje:
"What are your key skills as an assistant?"
Usa el mensaje del sistema definido por el usuario.


In [76]:

# Entrada del usuario para definir el mensaje del sistema
user_defined_role = input("Define the role of the assistant: ")
system_message = f"You are {user_defined_role.strip()}."

# Crear lista de mensajes
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "What are your key skills as an assistant?"}
]

# Enviar mensaje y recibir respuesta
response = send_message(messages, deployment_name)

messages.append({"role": "assistant", "content": response})

# Mostrar la conversación
print_conversation(messages)


[SYSTEM]
You are Cocinero top.

[USER]
What are your key skills as an assistant?

[ASSISTANT]
As an assistant, my key skills include:

1. **Information Retrieval**: I can quickly provide relevant information on a wide range of topics, from cooking techniques to historical facts.

2. **Recipe Development**: I can suggest, modify, and create recipes based on specific ingredients, dietary preferences, or cuisine types.

3. **Meal Planning**: I can help plan balanced meals, suggest grocery lists, and provide tips for efficient cooking.

4. **Time Management**: I can assist in organizing cooking schedules, making meal prep more efficient.

5. **Problem-Solving**: I'm equipped to troubleshoot cooking issues, offering solutions for common kitchen problems.

6. **Nutritional Guidance**: I can provide insights into the nutritional value of ingredients and suggest healthier alternatives.

7. **Creativity**: I can inspire creativity in cooking by suggesting unique flavor pairings or presentation 

## Ejercicio 4: Crear un Generador de Respuestas Creativo
Enunciado:
Diseña un programa que permita configurar la creatividad del modelo (temperatura) a través de una entrada del usuario. Una vez configurado, responde a la pregunta:
"Can you suggest an out-of-the-box idea for a startup?"



In [87]:

# Entrada del usuario para definir la creatividad
temperature = float(input("Set the creativity level (0.0 - 1.0): "))
if not (0.0 <= temperature <= 1.0):
    raise ValueError("Creativity level must be between 0.0 and 1.0.")
max_response_tokens=1000

# Crear lista de mensajes
system_message = "You are a highly creative assistant."
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Can you suggest an out-of-the-box idea for a startup?"}
]

# Enviar mensaje con creatividad ajustada
response = send_message(messages, deployment_name, max_response_tokens, temperature)

# Mostrar la respuesta
print(response)

Absolutely! Here’s an out-of-the-box startup idea: **"Eco-Lockers: A Sustainable Shared Storage Solution."**

### Concept:
Eco-Lockers would be a network of smart, eco-friendly lockers installed in urban areas, specifically targeting city dwellers who need temporary storage for their belongings while they work, travel, or enjoy leisure activities. 

### Key Features:
1. **Sustainability Focus:** The lockers would be made from recycled materials and powered by renewable energy sources, promoting an eco-friendly approach.

2. **Smart Technology:** Users can access lockers via a mobile app, which offers features like real-time availability, booking, and electronic key access. The app also tracks usage patterns for optimizing locker placement.

3. **Community Sharing:** Users can rent out their personal storage space when they're not using it, creating a local sharing economy. This could include garages, spare rooms, or unused storage units.

4. **Subscription Model:** Offer a subscription

## Ejercicio 5: Cambiar el Tono del Mensaje
Enunciado:
Configura el mensaje del sistema para que ChatGPT responda en un tono humorístico y escribe una respuesta al mensaje:
"Explain quantum computing in a funny way."


In [83]:

# Mensaje del sistema humorístico
system_message = "You are a humorous assistant."
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Explain quantum computing in a funny way."}
]

# Enviar mensaje y recibir respuesta
response = send_message(messages, deployment_name)

#append assistant response
messages.append({"role": "assistant", "content": response})
print_conversation(messages)


[SYSTEM]
You are a humorous assistant.

[USER]
Explain quantum computing in a funny way.

[ASSISTANT]
Sure! Imagine you’re trying to find your way out of a giant maze packed with cheese. Classic computers are like a bunch of little mice individually running around, checking every path one at a time, squeaking their findings to each other. They’re efficient and get the job done, but it can take a while—especially if you’re stuck behind a particularly picky cheese-loving mouse who insists on zigzagging!

Now, quantum computers are like a group of supercharged, hyper-intelligent mice who decided to learn some tricky dance moves. Instead of running through the maze one by one, they perform a coordinated zig-zagging, flipping, and twirling dance that lets them explore multiple paths at once. It’s as if they’re in a competition to see who can find the cheese first while also making a TikTok video at the same time!

These quantum mice are using the weirdness of quantum mechanics—think Schrödi

## Ejercicio 6: Resumir Mensajes Previos
Enunciado:
Dado un historial de conversación de 5 mensajes, crea una función que resuma el contenido de los mensajes del usuario en una sola oración.



In [88]:

# Historial de mensajes
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell me about AI."},
    {"role": "user", "content": "How does machine learning work?"},
    {"role": "user", "content": "Can you explain neural networks?"},
    {"role": "user", "content": "What are the applications of deep learning?"}
]

# Resumir mensajes del usuario
def summarize_user_messages(messages):
    user_messages = [msg['content'] for msg in messages if msg['role'] == 'user']
    summary_prompt = "Summarize the following messages into one sentence: " + " ".join(user_messages)
    summary = send_message([{"role": "system", "content": "You are a helpful assistant."},
                            {"role": "user", "content": summary_prompt}], deployment_name)
    return summary

# Obtener y mostrar el resumen
summary = summarize_user_messages(messages)
print(f"User messages summary: {summary}")


User messages summary: AI encompasses various technologies, including machine learning, which uses algorithms to learn from data; neural networks, which are computational models inspired by the human brain; and deep learning, a subset of machine learning with applications in areas like image recognition, natural language processing, and autonomous systems.


## Ejercicio 7: Configuración de Temperatura y Tono
Enunciado:
Escribe un programa que cambie la configuración de temperatura a 0.9 para obtener respuestas más creativas y responde al mensaje:
"Suggest a unique title for a blog about AI in education."



In [90]:

# Mensajes iniciales
messages = [
    {"role": "system", "content": "You are a creative assistant."},
    {"role": "user", "content": "Suggest a unique title for a blog about AI in education."}
]

# temperatura
temperature = 0.9

# Ajustar temperatura y enviar mensaje
response = send_message(messages, deployment_name, max_response_tokens, temperature)

# Mostrar la respuesta
print(response)


"Learning Reimagined: The AI Revolution in Education"
